In [1]:

import pandas as pd
import os
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas_profiling

### Baseline Model

In [2]:
# set the path of the processed data
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
train_file_path = os.path.join(processed_data_path, "train.csv")
test_file_path = os.path.join(processed_data_path, "test.csv")

In [3]:
train_df = pd.read_csv(train_file_path, index_col="PassengerId")
test_df = pd.read_csv(test_file_path, index_col="PassengerId")

## Data preperation

In [4]:
prepped_train_data = train_df.loc[:, "Age":].as_matrix().astype("float")
prepped_train_test_data = train_df["Survived"].ravel()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [5]:
print(prepped_train_data.shape)
print(prepped_train_test_data.shape)

(891, 32)
(891,)


In [6]:


prepped_train, prepped_test, test_train, test_test = \
train_test_split(prepped_train_data, prepped_train_test_data, test_size=0.2, random_state=0)
print(prepped_train.shape)
print(test_train.shape)
print(prepped_test.shape)
print(test_test.shape)

(712, 32)
(712,)
(179, 32)
(179,)


In [7]:
model_dummy = DummyClassifier(strategy="most_frequent", random_state=0)

In [8]:
model_dummy.fit(prepped_train, test_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [9]:
model_dummy.score(prepped_test, test_test)

0.6145251396648045

In [10]:
confusion_matrix(test_test, model_dummy.predict(prepped_test))

array([[110,   0],
       [ 69,   0]])

In [11]:
model_logreg_1 = LogisticRegression(random_state=0)

In [12]:
model_logreg_1.fit(prepped_train, test_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
model_logreg_1.score(prepped_test, test_test)

0.8268156424581006

In [14]:
confusion_matrix(test_test, model_logreg_1.predict(prepped_test))

array([[95, 15],
       [16, 53]])

In [15]:
def get_subbmission_file(model, filename):
    #converting to the matrix
    test_X = test_df.as_matrix().astype("float")
    # make prediction
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({"PassengerId": test_df.index, "Survived": predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, "data", "external")
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to the file
    df_submission.to_csv(submission_file_path, index=False)

In [16]:
get_subbmission_file(model_logreg_1, "02_lr.csv")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
model_logreg_1.coef_

array([[-0.02923553,  0.00451899, -0.50687734,  0.66961288,  0.        ,
         0.10012052, -0.18832707, -0.41398124,  0.51092837,  1.08121044,
         0.38318919, -0.19260299, -0.33615771,  0.89986506,  0.43281204,
        -0.38829759,  0.37731842,  0.91161511,  0.98600282, -1.81358953,
         1.46349644, -0.3356932 , -0.64477056,  0.1166403 ,  0.19947119,
         0.24640572,  0.3818623 ,  0.44346438,  0.40641003,  0.0945051 ,
         0.30654211,  0.63783741]])

In [18]:
parameters = {"C": [1.0, 10.0, 50.0, 100.0, 1000.0], "penalty": ["l1","l2"]}
clf = GridSearchCV(model_logreg_1, param_grid=parameters, cv=3)

In [19]:
clf.fit(prepped_train, test_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [21]:
clf.best_score_

0.8300561797752809

In [22]:
clf.score(prepped_test, test_test)

0.8268156424581006

In [23]:
get_subbmission_file(clf, "03_lr.csv")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


### Feature Normalization and Standardization

#### Feature Normalization 

In [24]:
# feature normalization
scaler = MinMaxScaler()
x_trained_scaled = scaler.fit_transform(prepped_train)

In [25]:
x_trained_scaled[:,0].min(), x_trained_scaled[:,0].max()

(0.0, 1.0)

In [26]:
x_test_scaled = scaler.transform(prepped_test)

#### Feature Stanadardization

In [28]:
# feature standardization
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(prepped_train)
x_test_scaled = scaler.transform(prepped_test)

#### Create model after standardization

In [29]:
# base model
model_lr = LogisticRegression(random_state=0)
parameteres = {"C": [1.0, 10.0, 100.0, 1000.0], "penalty": ["l1", "l2"]}
clf = GridSearchCV(model_lr, param_grid=parameteres, cv=3)
clf.fit(x_train_scaled, test_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
clf.best_score_

0.8146067415730337

## Basic Profiling

In [31]:
pandas_profiling.ProfileReport(train_df)

Number of variables,34
Number of observations,891
Total Missing (%),0.0%
Total size in memory,236.8 KiB
Average record size in memory,272.1 B
Numeric,4
Categorical,0
Boolean,29
Date,0
Text (Unique),0
Rejected,1


## Model Persistence

In [43]:
import pickle

In [44]:
# create the file path
model_file_path = os.path.join(os.path.pardir, "models", "lr_model.pkl")
scaler_file_path = os.path.join(os.path.pardir, "models", "lr_scaler.pkl")

In [45]:
# open the files to write
model_file_pickle = open(model_file_path, "wb")
scaler_file_pickle = open(scaler_file_path, "wb")

In [46]:
# persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [47]:
model_file_pickle.close()
scaler_file_pickle.close()

### load the presisted file

In [50]:
# open files in read mode
model_file_pickle = open(model_file_path, "rb")
scaler_file_pickle = open(scaler_file_path, "rb")

#load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)

# close files
model_file_pickle.close()
scaler_file_pickle.close()

In [51]:
clf_loaded

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [52]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [54]:
X_test_scaled = scaler_loaded.transform(prepped_test)
clf_loaded.score(X_test_scaled, test_test)

0.8435754189944135